In [ ]:
import numpy as np
import cv2 

# create videocapture
cap = cv2.VideoCapture('output.mp4')

# create Background Subtractor objects
backSub = cv2.createBackgroundSubtractorKNN()

# Read Map Image
fmap = cv2.imread('2D_field.png')
fmap = cv2.resize(fmap,(1050,680))

# 9 correspondences for perspective transform
points1 = np.array([(144,166),
                    (1136,116),
                    (873,780),
                    (639,110),
                    (673,200),
                    (490,210),
                    (857,192),
                    (660,162),
                    (692,251)], dtype=np.int32)

points2 = np.array([(164,147),
                    (886,147),
                    (525,676),
                    (525,4),
                    (525,340),
                    (430,340),
                    (618,340),
                    (525,250),
                    (525,430)],   dtype=np.int32)   

# find perspective transform
H, _ = cv2.findHomography(points1, points2, cv2.RANSAC,5.0)

u=0 
# b=-1

while True:
#     b+=1
#     if b%400==0:
    
        u += 1
        # read video frame by frame
        ret, frame = cap.read()

        if ret == False:
            break



        # apply GaussianBlur 
        m = 5
        frame2 = cv2.GaussianBlur(frame,(m,m),0)
#         cv2.line(frame2,(0,120),(1280,57),color=(0,0,0),thickness=40)

        # update the background model (get foreground mask)
        fgMask = backSub.apply(frame2)

        # threshold for removing shadows
        ret, fgMask = cv2.threshold(fgMask,128,255,cv2.THRESH_BINARY)

        # opening
        # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,15))

        kernel = np.array([[0,0,0,1,1,0,0,0],
                            [0,0,0,1,1,0,0,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,0,1,1,1,1,0,0],
                            [0,0,1,1,1,1,0,0],
                            [0,0,1,1,1,1,0,0]], dtype='uint8')

        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)


        #closing
        kernel = np.array([ [0,0,0,1,1,0,0,0],
                            [0,0,0,1,1,0,0,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,1,1,1,1,1,1,0],
                            [0,0,1,1,1,1,0,0],
                            [0,0,1,1,1,1,0,0],
                            [0,0,1,1,1,1,0,0]], dtype='uint8')

        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)

        n,C,stats, centroids = cv2.connectedComponentsWithStats(fgMask);

        f = fmap.copy()


        for i in range(1,n):

            if stats[i,cv2.CC_STAT_TOP]<=240:
                if stats[i,cv2.CC_STAT_AREA]>20:
                    point = centroids[i].reshape(-1,1,2).astype(np.float32)
                    dst2 = cv2.perspectiveTransform(point,H).reshape(1,2)
                    dst2 = dst2.astype('int32')
                    cv2.circle(f,(dst2[0,0], dst2[0,1]), radius=7, color=[0,0,255], thickness=-1)
#                     box = frame[stats[i,1]:stats[i,1]+stats[i,3],stats[i,0]:stats[i,0]+stats[i,2]]
#                     box = cv2.resize(box,(45,45))
#                     filename = str(u)+str(i)+'.jpg'
#                     cv2.imwrite(filename,box)


            elif 240<stats[i,cv2.CC_STAT_TOP]<=480:
                if stats[i,cv2.CC_STAT_AREA]>400:
                    point = centroids[i].reshape(-1,1,2).astype(np.float32)
                    dst2 = cv2.perspectiveTransform(point,H).reshape(1,2)
                    dst2 = dst2.astype('int32')
                    cv2.circle(f,(dst2[0,0], dst2[0,1]), radius=7, color=[0,0,255], thickness=-1)
#                     box = frame[stats[i,1]:stats[i,1]+stats[i,3],stats[i,0]:stats[i,0]+stats[i,2]]
#                     box = cv2.resize(box,(45,45))
#                     filename = str(u)+str(i)+'.jpg'
#                     cv2.imwrite(filename,box)

            elif 480<stats[i,cv2.CC_STAT_TOP]:
                if stats[i,cv2.CC_STAT_AREA]>1600:
                    point = centroids[i].reshape(-1,1,2).astype(np.float32)
                    dst2 = cv2.perspectiveTransform(point,H).reshape(1,2)
                    dst2 = dst2.astype('int32')
                    cv2.circle(f,(dst2[0,0], dst2[0,1]), radius=7, color=[0,0,255], thickness=-1)
#                     box = frame[stats[i,1]:stats[i,1]+stats[i,3],stats[i,0]:stats[i,0]+stats[i,2]]
#                     box = cv2.resize(box,(45,45))
#                     filename = str(u)+str(i)+'.jpg'
#                     cv2.imwrite(filename,box)

        #r = cv2.selectROI("Image", frame)
        #imCrop = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
        # show the current frame ,the fg masks and Map
            cv2.imshow('Frame', frame)
            cv2.imshow('FG Mask', fgMask)
        #cv2.imshow('Map', imCrop)

            if cv2.waitKey(1) == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()    


error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-inblc7p7\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
b

5756000

In [ ]:
import numpy as np
import cv2 

# create videocapture
cap = cv2.VideoCapture('output.mp4')

# create Background Subtractor objects
backSub = cv2.createBackgroundSubtractorKNN()

# Read Map Image
fmap = cv2.imread('2D_field.png')
fmap = cv2.resize(fmap,(1050,680))

# 9 correspondences for perspective transform
points1 = np.array([(144,166),
                    (1136,116),
                    (873,780),
                    (639,110),
                    (673,200),
                    (490,210),
                    (857,192),
                    (660,162),
                    (692,251)], dtype=np.int32)

points2 = np.array([(164,147),
                    (886,147),
                    (525,676),
                    (525,4),
                    (525,340),
                    (430,340),
                    (618,340),
                    (525,250),
                    (525,430)],   dtype=np.int32)   

# find perspective transform
H, _ = cv2.findHomography(points1, points2, cv2.RANSAC,5.0)

u=0 

while True:
    
    u += 1
    # read video frame by frame
    ret, frame = cap.read()
    
    if ret == False:
        break
    
    
    
    # apply GaussianBlur 
    m = 5
    frame2 = cv2.GaussianBlur(frame,(m,m),0)
    cv2.line(frame2,(0,120),(1280,57),color=(0,0,0),thickness=40)

    # update the background model (get foreground mask)
    fgMask = backSub.apply(frame2)
    
    # threshold for removing shadows
    ret, fgMask = cv2.threshold(fgMask,128,255,cv2.THRESH_BINARY)
    
    # opening
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,15))
    
    kernel = np.array([[0,0,0,1,1,0,0,0],
                        [0,0,0,1,1,0,0,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,0,1,1,1,1,0,0],
                        [0,0,1,1,1,1,0,0],
                        [0,0,1,1,1,1,0,0]], dtype='uint8')
    
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    
    
    #closing
    kernel = np.array([ [0,0,0,1,1,0,0,0],
                        [0,0,0,1,1,0,0,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,1,1,1,1,1,1,0],
                        [0,0,1,1,1,1,0,0],
                        [0,0,1,1,1,1,0,0],
                        [0,0,1,1,1,1,0,0]], dtype='uint8')
    
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)
    
    n,C,stats, centroids = cv2.connectedComponentsWithStats(fgMask);
    
    f = fmap.copy()

   
    for i in range(1,n):
    
        if stats[i,cv2.CC_STAT_TOP]<=240:
            if stats[i,cv2.CC_STAT_AREA]>20:
                point = centroids[i].reshape(-1,1,2).astype(np.float32)
                dst2 = cv2.perspectiveTransform(point,H).reshape(1,2)
                dst2 = dst2.astype('int32')
                cv2.circle(f,(dst2[0,0], dst2[0,1]), radius=7, color=[0,0,255], thickness=-1)
                box = frame[stats[i,1]:stats[i,1]+stats[i,3],stats[i,0]:stats[i,0]+stats[i,2]]
                box = cv2.resize(box,(45,45))
                filename = str(u)+str(i)+'.jpg'
                cv2.imwrite(filename,box)
                
                
        elif 240<stats[i,cv2.CC_STAT_TOP]<=480:
            if stats[i,cv2.CC_STAT_AREA]>400:
                point = centroids[i].reshape(-1,1,2).astype(np.float32)
                dst2 = cv2.perspectiveTransform(point,H).reshape(1,2)
                dst2 = dst2.astype('int32')
                cv2.circle(f,(dst2[0,0], dst2[0,1]), radius=7, color=[0,0,255], thickness=-1)
                box = frame[stats[i,1]:stats[i,1]+stats[i,3],stats[i,0]:stats[i,0]+stats[i,2]]
                box = cv2.resize(box,(45,45))
                filename = str(u)+str(i)+'.jpg'
                cv2.imwrite(filename,box)
                
        elif 480<stats[i,cv2.CC_STAT_TOP]:
            if stats[i,cv2.CC_STAT_AREA]>1600:
                point = centroids[i].reshape(-1,1,2).astype(np.float32)
                dst2 = cv2.perspectiveTransform(point,H).reshape(1,2)
                dst2 = dst2.astype('int32')
                cv2.circle(f,(dst2[0,0], dst2[0,1]), radius=7, color=[0,0,255], thickness=-1)
                box = frame[stats[i,1]:stats[i,1]+stats[i,3],stats[i,0]:stats[i,0]+stats[i,2]]
                box = cv2.resize(box,(45,45))
                filename = str(u)+str(i)+'.jpg'
                cv2.imwrite(filename,box)
    
    #r = cv2.selectROI("Image", frame)
    #imCrop = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    # show the current frame ,the fg masks and Map
    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgMask)
    #cv2.imshow('Map', imCrop)

    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()    


error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-inblc7p7\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
import cv2 
cap = cv2.VideoCapture(0)
while 1:
    if ret == False:
        break
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
    
ret, frame = cap.read()
cap.release()
cv2.destroyAllWindows()  